In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/급성_심근경색_환자_수_시도별_월별__20231122140252.csv', encoding='ansi')

In [3]:
df.head()

,지역별(1),2014,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,...,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
0,지역별(1),1월,2월,3월,4월,5월,6월,7월,8월,9월,...,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
1,서울,505,509,498,475,534,444,421,471,449,...,605,621,643,562,559,585,591,638,655,527
2,부산,134,118,143,136,126,129,138,129,140,...,196,208,193,207,189,231,197,189,211,213
3,대구,237,194,243,220,245,234,231,211,220,...,252,219,216,235,220,213,234,224,222,223
4,인천,95,100,100,126,88,74,104,99,84,...,174,166,196,191,174,165,155,188,157,168


In [4]:
df.to_csv('data/test/심근경색.csv', index=False)

In [5]:
# '지역별(1)' 컬럼에서 '지역별(1)' 문자값을 가진 행을 제거
df = df[df['지역별(1)'] != '지역별(1)']

In [6]:
long_df = df.melt(id_vars=['지역별(1)'], 
                  var_name='STD_YYYYMM', 
                  value_name='심근경색')

In [7]:
long_df.head(3)

,지역별(1),STD_YYYYMM,심근경색
0,서울,2014,505
1,부산,2014,134
2,대구,2014,237


In [8]:
long_df.rename(columns={'지역별(1)': 'SIDO'}, inplace=True)

In [9]:

# 변경할 문자열 매핑
name_mapping = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '경기': '경기도',
    '세종': '세종특별자치시',
    '강원': '강원도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주특별자치도'
}

# 'SIDO' 컬럼의 값을 매핑에 따라 변경
long_df['SIDO'] = long_df['SIDO'].map(name_mapping)

In [10]:
# 'STD_YYYYMM' 열의 숫자가 아닌 문자를 제거
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].str.replace('[^\d]', '', regex=True)

In [11]:
# 결과를 정수 타입으로 변환
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].astype(int)

In [12]:
def increment_month_properly(date_int):
    date_str = str(date_int)
    # 년도만 있는 경우, 01월을 추가
    if len(date_str) == 4:
        return date_str + '01'
    # 년도와 한 자리 월이 있는 경우 (예: 20141)
    elif len(date_str) == 5:
        year, month = int(date_str[:4]), int(date_str[4:])
        new_month = month + 1 if month < 9 else 10  # 9월 이하면 1 증가, 그렇지 않으면 10월
        return f"{year}0{new_month}"
    # 년도와 두 자리 월이 있는 경우
    elif len(date_str) == 6:
        year, month = int(date_str[:4]), int(date_str[4:])
        # 월을 1 증가시키기 (12월인 경우 다음 해로 넘어가지 않음)
        new_month = month + 1 if month < 12 else month
        return f"{year}{new_month:02d}"
    else:
        return date_str  # 형식에 맞지 않는 경우 그대로 반환

long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].apply(increment_month_properly)


In [13]:
long_df['심근경색'].dtype

dtype('O')

In [14]:
# '심근경색' 컬럼을 수치형(float)으로 변환
long_df['심근경색'] = pd.to_numeric(long_df['심근경색'], errors='coerce')

# 누락된 값이 있는 행 찾기
missing_values_df = long_df[long_df['심근경색'].isna()]

# 'STD_YYYYMM' 별 '심근경색' 평균값 계산 및 소수점 첫째 자리까지 반올림
mean_values = long_df.groupby('STD_YYYYMM')['심근경색'].transform('mean').round(1)

# 누락된 '심근경색' 값에 평균값 할당
long_df['심근경색'] = long_df['심근경색'].fillna(mean_values)

In [15]:
long_df.to_csv('data/test/심근경색.csv', index=False)